In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.searchers.hierarchical_searcher import HierarchicalSearcher
from Qommunity.iterative_searcher import IterativeSearcher

In [2]:
G = nx.powerlaw_cluster_graph(n=100, m=1, p=0.1)
num_reads = 100
version = ""
region = "na-west-1"

### Usage

In [3]:
advantage = AdvantageSampler(
    G, num_reads=num_reads, version=version, region=region, use_clique_embedding=True, elapse_times=True, return_sampleset_metadata=True
)

In [4]:
iterative_searcher = IterativeSearcher(advantage)

In [5]:
res = iterative_searcher.run_with_sampleset_info(
    num_runs=2,
    save_results=True,
    saving_path="advantage_clique_samples",
    iterative_verbosity=1,
    return_sampleset_metadata=True,
    process_results=False
)

Starting community detection iterations


  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [01:42<01:42, 102.48s/it]

Iteration 0 completed


100%|██████████| 2/2 [03:37<00:00, 108.86s/it]

Iteration 1 completed


In [ ]:
res.

rec.array([(list([[0, 7, 16, 20, 34, 44, 45, 48, 51, 52, 53, 54, 57, 68, 81, 83, 86, 89, 91], [56, 75, 77], [25, 33, 58, 94], [11, 42, 82], [6, 8, 63, 64, 84, 92], [3, 50, 73, 95, 98], [2, 10, 19, 26, 29, 32, 35, 43, 60, 96], [4, 12, 22, 28, 40, 41, 49, 55, 62, 65, 69, 99], [5, 15, 27, 39, 59], [1, 9, 17, 23, 24, 30, 46, 47, 66, 93, 97], [13, 18, 71, 72, 74, 76, 79, 85], [14, 21, 31, 36, 37, 38, 61, 67, 70, 78, 80, 87, 88, 90]]), 0.77038057, 102.45857477, list([[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]], [[0, 2, 3, 6, 7, 8, 10, 11, 16, 19, 20, 25, 26, 29, 32, 33, 34, 35, 42, 43, 44, 45, 48, 50, 51, 52, 53, 54, 56, 57, 58, 60, 63, 64, 68, 73, 

In [50]:
res[0].dwave_sampleset_infos.qpu_access_time.sum()

697037.52

In [27]:
for run in res:
    communities = run.communities
    modularities = run.modularity
    div_trees = run.division_tree
    div_modularities = run.division_modularities
    dwave_sampleset_info = run.dwave_sampleset_infos
    times_measured = run.time_measurements

First run

In [61]:
res[0].time_measurements.find_clique_embedding_time

array([0.1414337, 0.3123298, 0.1391013, 0.3731998, 0.2271785, 0.2742287,
       0.1232225, 0.165031 , 0.1379719, 0.1458408, 0.1343016, 0.1420312,
       0.332811 , 0.100343 , 0.1122439, 0.2090429, 0.1224837, 0.123875 ,
       0.1597369, 0.1049684, 0.1861382, 0.1947471])

Total time of "find_clique_embedding":

In [32]:
res[0].time_measurements.find_clique_embedding_time.sum()

3.962260900036199

In [41]:
res[1].dwave_sampleset_infos.qpu_access_time

array([39833.56, 29476.36, 35895.16, 35666.76, 24015.16, 36644.76,
       28392.36, 35576.76, 28437.16, 28173.16, 29365.96, 36489.16,
       28173.56, 36057.16, 26389.96, 35666.76, 35894.76, 28421.16,
       28173.96, 24016.36, 22365.16, 22365.16, 28129.56, 28436.76])

### Heuristic

In [62]:
advantage = AdvantageSampler(
    G, num_reads=num_reads, version=version, region=region, use_clique_embedding=False, measure_times=True, return_sampleset_info=True
)

In [63]:
iterative_searcher = IterativeSearcher(advantage)

In [65]:
res_heu = iterative_searcher.run_with_sampleset_info(
    num_runs=2,
    save_results=True,
    saving_path="heuristic)advantage_clique_samples",
    iterative_verbosity=1,
    return_sampleset_info=True,
)

Starting community detection iterations


 50%|█████     | 1/2 [04:56<04:56, 296.86s/it]

Iteration 0 completed


100%|██████████| 2/2 [14:59<00:00, 449.94s/it]

Iteration 1 completed


In [71]:
res_heu[0].time_measurements.embedding_composite_time

array([0.2807632, 0.2406278, 0.3359402, 0.302822 , 0.1841993, 0.1517898,
       0.2029491, 0.320833 , 0.2347661, 0.1915635, 0.2737376, 0.2755818,
       0.1963269, 0.2202596, 0.3618938, 0.4035531, 0.4405374, 0.427501 ,
       0.3805251, 0.3727842, 0.3873692, 0.456362 , 0.4334511, 0.3993097])

In [72]:
res_heu[0].time_measurements

rec.array([(0.2807632, 59.6475311), (0.2406278, 16.8091243),
           (0.3359402,  7.4627778), (0.302822 ,  0.6165701),
           (0.1841993,  0.7094622), (0.1517898,  0.0663413),
           (0.2029491,  0.2232563), (0.320833 ,  0.1497798),
           (0.2347661,  0.1103587), (0.1915635,  0.7009316),
           (0.2737376,  0.2508542), (0.2755818,  0.1312484),
           (0.1963269,  0.0945934), (0.2202596, 38.5239291),
           (0.3618938,  4.9918162), (0.4035531,  0.2753941),
           (0.4405374,  2.4519983), (0.427501 ,  0.1697537),
           (0.3805251,  1.2791192), (0.3727842,  1.8661805),
           (0.3873692,  0.2791576), (0.456362 ,  0.1886594),
           (0.4334511,  0.1984597), (0.3993097,  0.5174587)],
          dtype=[('embedding_composite_time', '<f8'), ('sample_time', '<f8')])

In [74]:
res_heu[0].time_measurements.embedding_composite_time.sum()

7.4754465000587516

In [78]:
res_heu[0].time_measurements.sample_time.sum()

137.71475569996983

In [79]:
res_heu[0].time_measurements.sample_time

array([59.6475311, 16.8091243,  7.4627778,  0.6165701,  0.7094622,
        0.0663413,  0.2232563,  0.1497798,  0.1103587,  0.7009316,
        0.2508542,  0.1312484,  0.0945934, 38.5239291,  4.9918162,
        0.2753941,  2.4519983,  0.1697537,  1.2791192,  1.8661805,
        0.2791576,  0.1886594,  0.1984597,  0.5174587])

In [81]:
res_heu[0].time_measurements.embedding_composite_time

array([0.2807632, 0.2406278, 0.3359402, 0.302822 , 0.1841993, 0.1517898,
       0.2029491, 0.320833 , 0.2347661, 0.1915635, 0.2737376, 0.2755818,
       0.1963269, 0.2202596, 0.3618938, 0.4035531, 0.4405374, 0.427501 ,
       0.3805251, 0.3727842, 0.3873692, 0.456362 , 0.4334511, 0.3993097])